# Modeling
steps:
- Preprocessing the test set like the train data
- Scaling the data (test and train)
- function for model_evaluation
- function for model building
- 

In [33]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.metrics import fbeta_score, make_scorer, f1_score, accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


RSEED = 42

## Scaling the data

## Modeling

In [17]:
df = pd.read_csv('data/Train.csv')

In [19]:
X_train = df.drop(
    ['Place_ID X Date', 'Date', 'Place_ID', 
     'target', 'target_min', 'target_max', 
     'target_variance', 'target_count'],
    axis=1
)
y_train = df.target

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train)

In [23]:
X_train.isnull().sum()

precipitable_water_entire_atmosphere        0
relative_humidity_2m_above_ground           0
specific_humidity_2m_above_ground           0
temperature_2m_above_ground                 0
u_component_of_wind_10m_above_ground        0
                                        ...  
L3_CH4_aerosol_optical_depth            18584
L3_CH4_sensor_azimuth_angle             18584
L3_CH4_sensor_zenith_angle              18584
L3_CH4_solar_azimuth_angle              18584
L3_CH4_solar_zenith_angle               18584
Length: 74, dtype: int64

In [24]:
X_train.fillna(X_train.mean(), inplace=True)

In [11]:
# Beispiel
# testing of different models

# root mean squared error
# r^2


def model_evaluation(clf, X_train, y_train, scoring=(r2_score, mean_squared_error)):

    results = {}
    scores = cross_validate(clf, X_train, y_train, scoring=scoring, cv=5, n_jobs=-1)
    results = {key: [value.mean().round(4), value.std().round(4)] for key, value in scores.items()}

    #del results['fit_time']
    #del results['score_time']

    return results

In [36]:

list_of_clf = [LinearRegression(),
               KNeighborsRegressor(),
               RandomForestRegressor(),
               GradientBoostingRegressor()]


scorers = {
    'r2': 'r2',
    'rmse': make_scorer(mean_squared_error, greater_is_better=False, squared=False)
}

def model_evaluation(estimator, scoring, X, y, cv=5):
    return cross_validate(estimator, X, y, cv=cv, scoring=scoring, return_train_score=False)

for reg in list_of_clf:
    results = model_evaluation(reg, scorers, X_train, y_train, cv=5)
    print(reg)
   
    r2_mean, r2_std = results['test_r2'].mean(), results['test_r2'].std()
    rmse_mean, rmse_std = (-results['test_rmse']).mean(), (-results['test_rmse']).std()  # Vorzeichen umdrehen
    print(f'R²   (mean ± std): {r2_mean:.3f} ± {r2_std:.3f}')
    print(f'RMSE (mean ± std): {rmse_mean:.3f} ± {rmse_std:.3f}')
    print('----'*10)

LinearRegression()
R²   (mean ± std): 0.370 ± 0.028
RMSE (mean ± std): 37.227 ± 1.170
----------------------------------------
KNeighborsRegressor()
R²   (mean ± std): 0.144 ± 0.019
RMSE (mean ± std): 43.410 ± 0.998
----------------------------------------
RandomForestRegressor(random_state=42)
R²   (mean ± std): 0.573 ± 0.024
RMSE (mean ± std): 30.642 ± 1.229
----------------------------------------
GradientBoostingRegressor()
R²   (mean ± std): 0.482 ± 0.023
RMSE (mean ± std): 33.775 ± 1.041
----------------------------------------


In [32]:
# Beispiel
list_of_clf = [LinearRegression(), KNeighborsClassifier(), RandomForestClassifier(random_state=RSEED)]

for clf in list_of_clf:
    results = model_evaluation(clf, (r2_score, mean_squared_error), X_train, y_train)
    print(clf)
    print('R^2 (mean, std): ', results["test_r2"])
    print('RMSE (mean, std): ', -results["test_neg_mean_squared_error"])
    print('----'*10)

ValueError: The list/tuple elements must be unique strings of predefined scorers.  One or more of the elements were callables. Use a dict of score name mapped to the scorer callable. Got (<function r2_score at 0x1295151c0>, <function mean_squared_error at 0x129514ea0>)

## Hyperparameter Tuning

- LinearRegression - GradientDescent
- KNN - GridSearchCV?
- Decision Tree - GridSearchCV?

## Evaluation

of different models

## Error Analysis